线性回归的从零开始实现，模型是**y** = **Xw** + b + θ，θ服从（0,0.0001）的正态分布。**X**是一个1000*2的矩阵，每个样本包含从标准正态分布中采样的2个特征，线性模型参数**W** = [2，-3.4]<sup>T</sup>，b = 4.2

In [1]:
import random
import torch

产生数据

In [2]:
def synthetic_data(w, b, num_examples):     #@save
    """ 生成线性回归的模型 """
    X = torch.normal(0, 1, (num_examples, len(w)))  # 返回一个1000*2的tensor，数据服从标准正态分布
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)  # 加θ
    return X, y.reshape((-1, 1))  # 返回X，y的列向量

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

print('features:',features[0],'\nlabel:',labels[0])  # features中的每一行都包含一个二维数据样本，labels中的每一行都包含一维标签值（一个标量）

features: tensor([ 1.3837, -1.3548]) 
label: tensor([11.5683])


读取数据集
该函数能打乱数据集集中的样本并以小批量方式获取数据（batch_size）

In [3]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))

    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]
    
batch_size = 10

for X,y in data_iter(batch_size, features, labels):
        print(X, '\n', y)
        break

tensor([[-0.3436,  1.0502],
        [ 1.4884,  1.1913],
        [ 0.3127,  1.3316],
        [ 0.1652,  0.8474],
        [-0.0790,  0.0616],
        [ 1.0388,  0.4678],
        [ 0.1514, -0.6172],
        [-0.6109,  0.9760],
        [-2.0698, -1.2011],
        [-0.3881, -0.6774]]) 
 tensor([[-0.0539],
        [ 3.1356],
        [ 0.3045],
        [ 1.6498],
        [ 3.8470],
        [ 4.6937],
        [ 6.5856],
        [-0.3517],
        [ 4.1269],
        [ 5.7190]])


初始化模型参数

In [4]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

初始化参数之后，我们的任务是更新这些参数，直到这些参数足够拟合我们的数据。更新的算法为梯度下降算法（gradient descent）

---------------
定义线性回归模型

In [5]:
def linreg(X, w, b):    #@save
    """ 线性回归模型 """
    return torch.matmul(X, w) + b

定义损失函数，这里使用平方损失函数

In [6]:
def squared_loss(y_hat, y): #@save
    """ 均方损失 """
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

定义优化算法，主要使用小批量随机梯度下降。

In [7]:
def sgd(params, lr, batch_size):  #@save
    """ 小批量随机梯度下降 """
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size   # 计算的损失是一个批量样本的总和，所以用batch_size来归一化步长
                                                    # -=操作符会就地改变参数的值，所以不需要返回值
            param.grad.zero_()

训练模型，我们会完成以下循环：
* 初始化参数
* 重复，直到完成
    * 计算梯度**g（w，** b **）**
    * 更新参数**w**，b

In [8]:
lr = 0.03
num_epochs = 3

""" 等于将前面定义的函数改了个名 """
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X,y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w,b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.036472
epoch 2, loss 0.000132
epoch 3, loss 0.000049


真实参数true_w，true_b与训练参数w，b的误差

In [9]:
print(f'w的估计误差:{true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差:{true_b - b}')

w的估计误差:tensor([ 0.0010, -0.0003], grad_fn=<SubBackward0>)
b的估计误差:tensor([-0.0001], grad_fn=<RsubBackward1>)
